##  二维向量类 版本1

In [1]:
from array import array
import math
import reprlib

In [2]:
class Vector2d:
    typecode = 'd'
    __slots__ = ('__x', '__y') # 使用__slots__节省空间
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    # 只读属性    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
            
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [3]:
a = Vector2d(1, 2)

In [4]:
a.x, a.y

(1.0, 2.0)

## 创建序列化可切片多维向量类 版本2

In [5]:
from array import array


class VectorNd:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __len__(self):
        return len(self._components)
    
    # 切片的入口
    def __getitem__(self, index):
        return self._components[index]
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'VectorNd({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))

In [6]:
a = VectorNd(range(10))
a

VectorNd([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [7]:
a[1:4]

array('d', [1.0, 2.0, 3.0])

In [8]:
a[-1]

9.0

但切片返回的是array，不是VectorNd

## 正确返回切片类型 版本3

In [9]:
from array import array
import numbers

class VectorNd:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __len__(self):
        return len(self._components)
    
    # 对传进来的index进行处理，返回正确的切片
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'VectorNd({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))

In [10]:
a = VectorNd(range(10))
a

VectorNd([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [11]:
a[1:5]

VectorNd([1.0, 2.0, 3.0, 4.0])

In [12]:
a[2]

2.0

## 动态存取属性 版本4

版本2、3中，不能使用名称访问向量的分量（如 v.x 和 v.y）。现在我们处理的向量可能有大量分量。不过，若能通过单个字母访问前几个分量的话会比较方便。比如，用 `x`、 `y` 和 `z` 代替 `v[0]`、 `v[1]` 和 `v[2]`。

在 `Vector2d` 中，我们使用 `@property` 装饰器把 `x` 和 `y` 标记为只读特性。我们可以在 Vector 中编写三个特性，但这样太麻烦。特殊方法 `__getattr__` 提供更好的方式。

属性查找失败后，解释器会调用 `__getattr__` 方法。简单来说，对 `my_obj.x` 表达式，Python 会检查 `my_obj` 实例有没有名为 `x` 的属性；如果没有，到类`（my_obj.__class__）`中查找；如果还没有，顺着继承树继续查找。 如果依旧找不到，调用 `my_obj` 所属类中定义的 `__getattr__` 方法，传入 `self` 和属性名称的字符串
形式（如 `x`）。

In [13]:
from array import array
import numbers

class VectorNd:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __len__(self):
        return len(self._components)
    
    # 对传进来的index进行处理，返回正确的切片
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'VectorNd({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        return hash(self.x)^hash(self.y)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))

In [14]:
v = VectorNd(range(10))
v.__dict__

{'_components': array('d', [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0])}

In [15]:
v.x, v.y, v.z, v.t

(0.0, 1.0, 2.0, 3.0)

`__getattr__` 方法的实现不难，但是这样实现还不够。看看下面古怪的交互行为，赋值没有抛出错误，但是前后矛盾

In [16]:
v.x

0.0

In [17]:
v.x = 10

In [18]:
v.x

10

In [19]:
v

VectorNd([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [20]:
v.__dict__

{'_components': array('d', [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]),
 'x': 10}

之所以前后矛盾，是 `__getattr__` 的运作方式导致的：仅当对象没有指定名称的属性时， Python 才会调用那个方法，这是一种后备机制。可是，像 `v.x = 10` 这样赋值之后， `v` 对象有 `x` 属性了，因此使用 `v.x` 获取 `x` 属性的值时不会调用 `__getattr__` 方法了，解释器直接返回绑定到 `v.x` 上的值，即 10。另一方面， `__getattr__` 方法的实现没有考虑到 `self._components` 之外的实例属性，而是从这个属性中获shortcut_names 中所列的“虚拟属性”。为了避免这种前后矛盾的现象，我们要改写 Vector 类中设置属性的逻辑，加入 `__setattr__`。

In [21]:
from array import array
import numbers

class VectorNd:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 这里已经开始调用下面定义的__setattr__
        self._components = array(self.typecode, components)
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'randonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
                
        super().__setattr__(name, value)
        
    def __len__(self):
        return len(self._components)
    
    # 对传进来的index进行处理，返回正确的切片
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'VectorNd({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))

In [22]:
v = VectorNd(range(10))
v

VectorNd([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [23]:
v.x

0.0

因为我们在`__setattr__`中做了限制，所以不能对a-z赋值

In [24]:
v.x = 10

AttributeError: randonly attribute 'x'

多数时候，如果实现了 `__getattr__` 方法，那么也要定义 `__setattr__` 方法，以防对象的行为不一致。

如果想允许修改分量，可以使用 `__setitem__` 方法，支持 `v[0] = 1.1` 这样的赋值，以及（或者）实现 `__setattr__` 方法，支持 `v.x = 1.1` 这样的赋值。

## 实现散列和快速等值测试 版本5

In [25]:
from array import array
import numbers
import functools
import operator

class VectorNd:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 这里已经开始调用下面定义的__setattr__
        self._components = array(self.typecode, components)
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'randonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
                
        super().__setattr__(name, value)
        
    def __len__(self):
        return len(self._components)
    
    # 对传进来的index进行处理，返回正确的切片
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'VectorNd({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __eq__(self, other):
        # 这里没有用tuple(self) == tupe(other)，防止数据量大时效率低下
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))

In [26]:
v1 = VectorNd([1, 2, 3])
v2 = VectorNd([1.0, 2.0, 3.0])
v3 = VectorNd([1, 2, 4])

In [27]:
v1 == v2, v1 == v3

(True, False)

In [28]:
dt = {}
dt[v1] = 1
dt[v3] = 2
dt

{VectorNd([1.0, 2.0, 3.0]): 1, VectorNd([1.0, 2.0, 4.0]): 2}

In [29]:
dt[v2] += 1

In [30]:
dt

{VectorNd([1.0, 2.0, 3.0]): 2, VectorNd([1.0, 2.0, 4.0]): 2}